In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import(precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, mean_absolute_error)
from sklearn.preprocessing import Normalizer
from sklearn import model_selection as ms
from sklearn import preprocessing as pp
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers import Dense, Dropout,Flatten, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import h5py

In [ ]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Deep CNN Model/results/cnn_deep_mawi2.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [ ]:
# get information from MAWILab Dataset

class dataset:
    mawi_train_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels(001).pkl")
    mawi_test_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels(003).pkl")

class preprocess:
    output_columns_2labels = ['status_anomaly','status_normal']

    x_input = dataset.mawi_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.mawi_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.mawi_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.mawi_test_2labels.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (40934, 4)
y_train shape: (40934, 2)
x_test shape: (42398, 4)
y_test shape: (42398, 2)


In [ ]:
#reshape data
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (40934, 4, 1)
test shape after reshape: (42398, 4, 1)


In [ ]:
#size of parameters
batch_size = 64
num_classes = 2
epochs = 20
filter_size=3
droprate=0.50

In [ ]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(4, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))

#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), padding="same",
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#convolution 3rd layer
model.add(Conv1D(256, kernel_size=(filter_size), padding="same",
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#Flatten
model.add(Flatten())

#FCN Layer
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN Layer
model.add(Dense(32,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN Layer
model.add(Dense(16,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 4, 64)             256       
                                                                 
 batch_normalization_12 (Bat  (None, 4, 64)            256       
 chNormalization)                                                
                                                                 
 dropout_12 (Dropout)        (None, 4, 64)             0         
                                                                 
 conv1d_7 (Conv1D)           (None, 4, 128)            24704     
                                                                 
 batch_normalization_13 (Bat  (None, 4, 128)           512       
 chNormalization)                                                
                                                                 
 dropout_13 (Dropout)        (None, 4, 128)           

In [ ]:
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 40934 samples, validate on 42398 samples
Epoch 1/20
40896/40934 [============================>.] - ETA: 0s - loss: 0.5127 - acc: 0.7765

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


40934/40934 [==============================] - 14s 353us/sample - loss: 0.5125 - acc: 0.7766 - val_loss: 0.5363 - val_acc: 0.6717
Epoch 2/20
40934/40934 [==============================] - 12s 303us/sample - loss: 0.3966 - acc: 0.8249 - val_loss: 0.5294 - val_acc: 0.6717
Epoch 3/20
40934/40934 [==============================] - 12s 289us/sample - loss: 0.3834 - acc: 0.8253 - val_loss: 0.5347 - val_acc: 0.6717
Epoch 4/20
40934/40934 [==============================] - 12s 302us/sample - loss: 0.3784 - acc: 0.8254 - val_loss: 0.5409 - val_acc: 0.6717
Epoch 5/20
40934/40934 [==============================] - 12s 302us/sample - loss: 0.3758 - acc: 0.8252 - val_loss: 0.5433 - val_acc: 0.6717
Epoch 6/20
40934/40934 [==============================] - 12s 304us/sample - loss: 0.3743 - acc: 0.8263 - val_loss: 0.5443 - val_acc: 0.6789
Epoch 7/20
40934/40934 [==============================] - 13s 307us/sample - loss: 0.3717 - acc: 0.8274 - val_loss: 0.5364 - val_acc: 0.6781
Epoch 8/20
40934/40934 [

In [ ]:
#Confusion Matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)

target_name=["is_Attack", "is_Normal"]

print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_name, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

   is_Attack     0.5864    0.0914    0.1581     13921
   is_Normal     0.6856    0.9685    0.8028     28477

    accuracy                         0.6805     42398
   macro avg     0.6360    0.5299    0.4805     42398
weighted avg     0.6530    0.6805    0.5911     42398

[[ 1272 12649]
 [  897 27580]]
